# HPC Performance Analysis
Plotting BLIS and OpenBLAS results and saving graphs to `analysis/` folder


In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create analysis folder if it doesn't exist
os.makedirs("/Users/manasamaddi/Documents/HPC/Analysis", exist_ok=True)


In [13]:
# Defining the base paths

DATA_ROOT = "/Users/manasamaddi/Documents/HPC/HPC_data"
OUTPUT_DIR = "/Users/manasamaddi/Documents/HPC/Analysis"

In [14]:
# Cell 2: Plot function for "Fixed Cores, Variable Matrix Size"
def plot_fixed_cores(node_kind, precision, tpp):
    res_path = os.path.join(DATA_ROOT, "fixed_cores", node_kind)
    suffix = f"_{precision}"
    column_names = ['m', 'k', 'n', 'time', 'gflops']
    
    def load_and_avg(filepath):
        if not os.path.exists(filepath):
            print(f"Warning: {filepath} not found. Skipping.")
            return None
        df = pd.read_csv(filepath, names=column_names, header=0)
        return df.groupby("m")[['gflops', 'time']].mean().reset_index()  # ← FIXED
    
    blis_close = load_and_avg(os.path.join(res_path, "close_cores", f"blis{suffix}.csv"))
    openblas_close = load_and_avg(os.path.join(res_path, "close_cores", f"openblas{suffix}.csv"))
    blis_spread = load_and_avg(os.path.join(res_path, "spread_cores", f"blis{suffix}.csv"))
    openblas_spread = load_and_avg(os.path.join(res_path, "spread_cores", f"openblas{suffix}.csv"))
    
    plt.figure(figsize=(8, 6))
    
    if blis_close is not None:
        plt.plot(blis_close['m'], blis_close['gflops'], label="BLIS Close", color='blue')
    if openblas_close is not None:
        plt.plot(openblas_close['m'], openblas_close['gflops'], label="OpenBLAS Close", color='dodgerblue')
    if blis_spread is not None:
        plt.plot(blis_spread['m'], blis_spread['gflops'], label="BLIS Spread", color='forestgreen')
    if openblas_spread is not None:
        plt.plot(openblas_spread['m'], openblas_spread['gflops'], label="OpenBLAS Spread", color='limegreen')
    
    # Add TPP line if any data exists
    if any(x is not None for x in [blis_close, blis_spread]):
        m_vals = (blis_close['m'] if blis_close is not None else blis_spread['m'])
        plt.axhline(y=tpp, color='pink', linestyle='--', label="TPP")
    
    plt.xlabel("Matrix dimension (m)")
    plt.ylabel("Performance [Gflops/s]")
    plt.title(f"Fixed Cores – {node_kind} ({'float32' if precision == 'f' else 'float64'})")
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)
    
    output_file = os.path.join(OUTPUT_DIR, f"fixed_cores_{node_kind}_{precision}.png")
    plt.savefig(output_file, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {output_file}")


In [15]:
# Cell 3: Plot function for "Fixed Matrix Size, Variable Cores"
def plot_fixed_matrix(node_kind, precision, tpp):
    res_path = os.path.join(DATA_ROOT, "fixed_matrix", node_kind)
    suffix = f"_{precision}"
    column_names = ['cores', 'm', 'k', 'n', 'time', 'gflops']
    
    def load_and_avg(filepath):
        if not os.path.exists(filepath):
            print(f"Warning: {filepath} not found. Skipping.")
            return None
        df = pd.read_csv(filepath, names=column_names, header=0)
        return df.groupby("cores")[['gflops', 'time']].mean().reset_index()  # ← FIXED
    
    blis_close = load_and_avg(os.path.join(res_path, "close_cores", f"blis{suffix}.csv"))
    openblas_close = load_and_avg(os.path.join(res_path, "close_cores", f"openblas{suffix}.csv"))
    blis_spread = load_and_avg(os.path.join(res_path, "spread_cores", f"blis{suffix}.csv"))
    openblas_spread = load_and_avg(os.path.join(res_path, "spread_cores", f"openblas{suffix}.csv"))
    
    plt.figure(figsize=(8, 6))
    
    if blis_close is not None:
        plt.plot(blis_close['cores'], blis_close['gflops'], label="BLIS Close", color='blue')
    if openblas_close is not None:
        plt.plot(openblas_close['cores'], openblas_close['gflops'], label="OpenBLAS Close", color='dodgerblue')
    if blis_spread is not None:
        plt.plot(blis_spread['cores'], blis_spread['gflops'], label="BLIS Spread", color='forestgreen')
    if openblas_spread is not None:
        plt.plot(openblas_spread['cores'], openblas_spread['gflops'], label="OpenBLAS Spread", color='limegreen')
    
    # Add TPP line
    if any(x is not None for x in [blis_close, blis_spread]):
        cores_vals = (blis_close['cores'] if blis_close is not None else blis_spread['cores'])
        plt.plot(cores_vals, [tpp] * len(cores_vals), 'k--', color='pink', label="TPP")
    
    plt.xlabel("Number of cores")
    plt.ylabel("Performance [Gflops/s]")
    plt.title(f"Fixed Matrix – {node_kind} ({'float32' if precision == 'f' else 'float64'})")
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)
    
    output_file = os.path.join(OUTPUT_DIR, f"fixed_matrix_{node_kind}_{precision}.png")
    plt.savefig(output_file, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {output_file}")

In [16]:
# Cell 4: Run all plots
plot_fixed_cores("THIN", "f", tpp=1997 * 0.5 * 2)
plot_fixed_cores("THIN", "d", tpp=1997 * 0.5)
plot_fixed_cores("EPYC", "f", tpp=2.60 * 16 * 64 * 2)
plot_fixed_cores("EPYC", "d", tpp=2.60 * 16 * 64)

plot_fixed_matrix("THIN", "f", tpp=1997 / 24 * 2)
plot_fixed_matrix("THIN", "d", tpp=1997 / 24)
plot_fixed_matrix("EPYC", "f", tpp=2.60 * 16 * 2)
plot_fixed_matrix("EPYC", "d", tpp=2.60 * 16)

Saved: /Users/manasamaddi/Documents/HPC/Analysis/fixed_cores_THIN_f.png
Saved: /Users/manasamaddi/Documents/HPC/Analysis/fixed_cores_THIN_d.png
Saved: /Users/manasamaddi/Documents/HPC/Analysis/fixed_cores_EPYC_f.png
Saved: /Users/manasamaddi/Documents/HPC/Analysis/fixed_cores_EPYC_d.png


/var/folders/gb/kl7k_yd53t1b4gmt1y4x66y40000gp/T/ipykernel_9397/600367821.py:33: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  plt.plot(cores_vals, [tpp] * len(cores_vals), 'k--', color='pink', label="TPP")
/var/folders/gb/kl7k_yd53t1b4gmt1y4x66y40000gp/T/ipykernel_9397/600367821.py:33: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  plt.plot(cores_vals, [tpp] * len(cores_vals), 'k--', color='pink', label="TPP")
/var/folders/gb/kl7k_yd53t1b4gmt1y4x66y40000gp/T/ipykernel_9397/600367821.py:33: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  plt.plot(cores_vals, [tpp] * len(cores_vals), 'k--', color='pink', label="TPP")
/var/folders/gb/kl7k_yd53t1b4gmt1y4x6

Saved: /Users/manasamaddi/Documents/HPC/Analysis/fixed_matrix_THIN_f.png
Saved: /Users/manasamaddi/Documents/HPC/Analysis/fixed_matrix_THIN_d.png
Saved: /Users/manasamaddi/Documents/HPC/Analysis/fixed_matrix_EPYC_f.png
Saved: /Users/manasamaddi/Documents/HPC/Analysis/fixed_matrix_EPYC_d.png
